**Please note that the WAME implementation used is compatible with tensorflow 2.0.0 only.
Please run this notebook in an environment where tensorflow 2.0.0 is installed.**

In [ ]:
#This code is just to check what version of tensorflow
#is installed on your system

import tensorflow as tf
print(tf.__version__)

In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
#We define here the WAMEprop class, to be used later

from tensorflow.python.framework import ops
from tensorflow.python.keras.optimizer_v2 import optimizer_v2
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops


class WAMEprop(optimizer_v2.OptimizerV2):
	"""WAME optimizer.
	It is recommended to leave the parameters of this optimizer at their default values as these have been
	shown empircally to deliver good results (except the learning rate, which can be freely tuned).
	The algorithm has been adapated slightly from the original paper to replace \frac{1}{\theta} with \frac{1}{\sqrt{\theta}} after 
	speaking with the algorithm developers. 
	# Arguments
		learning_rate: float >= 0. Base learning rate.
		alpha: float >= 0. Decay rate of the exponentially weighted moving average.
		eta_plus: float > 0. Multiplicative term of the acceleration factor for the case of a positive gradient product.
		eta_minus: float > 0. Multiplicative term of the acceleration factor for the case of a negative gradient product.
		zeta_min: float > 0. Lower bounding value for the accerlation factor.
		zeta_max: float > 0. Upper bounding value for the acceleration factor.
		epsilon: float > 0. A very small fudge factor requried to avoid a possible division by zero error.
	# References
		- [wame: Training Convolutional Networks with Weight–wise Adaptive Learning Rates]
		(https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2017-50.pdf)
	"""

	def __init__(self, learning_rate=0.0001, alpha = 0.9, eta_plus = 1.2, eta_minus = 0.1, zeta_min = 0.01, zeta_max = 100, epsilon = 1e-11, **kwargs):
		super(WAMEprop, self).__init__(**kwargs)
		self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))
		self._set_hyper("alpha", alpha)
		self._set_hyper("eta_plus", eta_plus)
		self._set_hyper("eta_minus", eta_minus)
		self._set_hyper("zeta_min", zeta_min)
		self._set_hyper("zeta_max", zeta_max)
		self.epsilon = epsilon
 
	def _create_slots(self, var_list):
		for var in var_list:
			self.add_slot(var, "zetas")
			self.add_slot(var, "zeds")
			self.add_slot(var, "thetas")
			self.add_slot(var, "old_grads")

	def _prepare_local(self, var_device, var_dtype, apply_state):
		super(WAMEprop, self)._prepare_local(var_device, var_dtype, apply_state)

		alpha = array_ops.identity(self._get_hyper("alpha", var_dtype))
		eta_plus = array_ops.identity(self._get_hyper("eta_plus", var_dtype))
		eta_minus = array_ops.identity(self._get_hyper("eta_minus", var_dtype))
		zeta_max = array_ops.identity(self._get_hyper("zeta_max", var_dtype))
		zeta_min = array_ops.identity(self._get_hyper("zeta_min", var_dtype))
		apply_state[(var_device, var_dtype)].update(
			dict(
				epsilon=ops.convert_to_tensor_v2(self.epsilon, var_dtype),
				alpha=alpha,
				eta_plus = eta_plus,
				eta_minus = eta_minus,
				zeta_max = zeta_max,
				zeta_min = zeta_min,
				one_minus_alpha = 1 - alpha))

	def _resource_apply_dense(self, grad, var, apply_state=None):
		var_device, var_dtype = var.device, var.dtype.base_dtype
		coefficients = ((apply_state or {}).get((var_device, var_dtype))
						or self._fallback_apply_state(var_device, var_dtype))

		zeta =  self.get_slot(var, 'zetas')
		zed =  self.get_slot(var, 'zeds')
		theta =  self.get_slot(var, 'thetas')
		old_grad =  self.get_slot(var, 'old_grads')

		new_z = tf.where(
			math_ops.Equal(x = grad * old_grad, y = 0),
		zeta,
		tf.where(math_ops.Greater(x = grad * old_grad, y = 0),
			x = math_ops.Minimum(x = zeta * coefficients['eta_plus'], y = coefficients['zeta_max']),
			y = math_ops.Maximum(x = zeta * coefficients['eta_minus'], y = coefficients['zeta_min'])
		)
		)
			
		new_z = state_ops.assign(zeta, new_z, use_locking=self._use_locking)

		new_zed = (coefficients["alpha"] * zed)  + (coefficients["one_minus_alpha"]*new_z)
		new_zed = state_ops.assign(zed, new_zed, use_locking=self._use_locking)

		new_t = (coefficients["alpha"] * theta)  + (coefficients["one_minus_alpha"]*math_ops.square(grad))
		new_t = state_ops.assign(theta, new_t, use_locking=self._use_locking)

		var_t  = var - (coefficients["lr_t"] * new_zed * grad * (1/(math_ops.Sqrt(x = new_t) +coefficients["epsilon"])))

		old_grad = state_ops.assign(old_grad, grad, use_locking=self._use_locking)

		return state_ops.assign(var, var_t, use_locking=self._use_locking).op

	def _resource_apply_sparse(self, grad, var):
		raise NotImplementedError("Sparse gradient updates are not supported.")
	
	def get_config(self):
		config = super(WAMEprop, self).get_config()
		config.update({'learning_rate': self._serialize_hyperparameter("learning_rate"),
				  'alpha': self._serialize_hyperparameter("alpha"),
				  'eta_plus': self._serialize_hyperparameter("eta_plus"),
				  'eta_minus': self._serialize_hyperparameter("eta_minus"),
				  'zeta_min': self._serialize_hyperparameter("zeta_min"),
				  'zeta_max': self._serialize_hyperparameter("zeta_max") 
				  })
		return config

In [ ]:
#We define the functions to build and fit a model, to be used later

def build_model(train_data, n_units):
    model = Sequential()
    model.add(Dense(n_units, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(n_units, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

def fit_model(model, train_features, train_targets, test_features, test_targets, num_epochs):
    history = model.fit(train_features.to_numpy(), train_targets.to_numpy(), validation_data=(test_features.to_numpy(), test_targets.to_numpy()),
                        epochs=num_epochs, batch_size=1, verbose=0)
    return history

def build_model_wame(train_data, n_units, lambda_rate=0.0001):
    model = Sequential()
    model.add(Dense(n_units, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(n_units, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=WAMEprop(name='wame', lr=lambda_rate), loss='mse', metrics=['mae'])
    return model

In [ ]:
path_white = input('''
                   Please paste here the complete path
                   to the white wine csv file:
                   ''' 
                   )
white_df = pd.read_csv(path_white, sep=';')

In [ ]:
#check on the dataframe shape

white_df.shape

In [ ]:
#glimpse at the data

white_df.head()

In [ ]:
#We separate the features form the target value,
#to pass them the Keras model.fit() method 

white_features_df = white_df.drop('quality', axis=1)
white_targets = white_df['quality'] #Note this is a series

In [ ]:
#This code executes the K-Fold cross validation to establish a sensible
#number of epochs to avoid overfitting

k_fold_validator = KFold(n_splits=10)
scaler = StandardScaler()

units_number = 32
epochs_number = 100

all_folds_mae_history = []
#this list will contain other lists, each of which will contain all the mae value for each epoch, for a certain fold

count = 0

for train_indices, test_indices in k_fold_validator.split(white_features_df):
    
  white_train_features_df = white_features_df.iloc[train_indices, :]
  white_train_features_df_s = pd.DataFrame(scaler.fit_transform(white_train_features_df),
                                             columns=white_train_features_df.columns)

  white_train_targets = white_targets[train_indices]
    
  white_test_features_df = white_features_df.iloc[test_indices, :]
  white_test_features_df_s = pd.DataFrame(scaler.fit_transform(white_test_features_df),
                                             columns=white_test_features_df.columns)

  white_test_targets = white_targets[test_indices]

  model = build_model(white_train_features_df_s, units_number)
  history = fit_model(model, white_train_features_df_s, white_train_targets, white_test_features_df_s, white_test_targets, epochs_number)
  history_dict = history.history
  fold_mae_history = history_dict['val_mae'] # this list contains the value of mae for each epoch, for this fold
  all_folds_mae_history.append(fold_mae_history)
  count += 1
  print('Fold ' + str(count) + ' done.')

In [ ]:
#To get the k-fold cross-validation values of MAE for each epoch
#we need to average those for each fold

average_mae_history = [np.mean([x[j] for x in all_folds_mae_history]) 
                       for j in range(epochs_number)]

#This code plots the result of the 10-fold cross validation
#with 100 epochs

fig, ax = plt.subplots()
ax.plot(range(epochs_number), average_mae_history)
ax.set_xlabel('Epochs')
ax.set_ylabel('Mean absolute error')

In [ ]:
#We run once again the cross-validation procedure,
#but limiting the number of epochs to 20, given the results above

k_fold_validator = KFold(n_splits=10)
scaler = StandardScaler()

units_number = 32
epochs_number = 20

all_folds_mae_history = []
#this list will contain other lists, each of which will contain all the mae value for each epoch, for a certain fold

count = 0

for train_indices, test_indices in k_fold_validator.split(white_features_df):
    
  white_train_features_df = white_features_df.iloc[train_indices, :]
  white_train_features_df_s = pd.DataFrame(scaler.fit_transform(white_train_features_df),
                                             columns=white_train_features_df.columns)

  white_train_targets = white_targets[train_indices]
    
  white_test_features_df = white_features_df.iloc[test_indices, :]
  white_test_features_df_s = pd.DataFrame(scaler.fit_transform(white_test_features_df),
                                             columns=white_test_features_df.columns)

  white_test_targets = white_targets[test_indices]

  model = build_model(white_train_features_df_s, units_number)
  history = fit_model(model, white_train_features_df_s, white_train_targets, white_test_features_df_s, white_test_targets, epochs_number)
  history_dict = history.history
  fold_mae_history = history_dict['val_mae'] # this list contains the value of mae for each epoch, for this fold
  all_folds_mae_history.append(fold_mae_history)
  count += 1
  print('Fold ' + str(count) + ' done.')

In [ ]:
#we plot the results once again, but for 20 epochs

average_mae_history = [np.mean([x[j] for x in all_folds_mae_history]) 
                       for j in range(epochs_number)]

fig, ax = plt.subplots()
ax.plot(range(epochs_number), average_mae_history)
ax.set_xlabel('Epochs')
ax.set_ylabel('Mean absolute error')

In [ ]:
#We now follow the same procedure as before, using the WAMEprop optimiser,
#to find the optimal number of epochs

k_fold_validator = KFold(n_splits=10)
scaler = StandardScaler()

units_number = 32
epochs_number = 200

all_folds_mae_history = []
#this list will contain other lists, each of which will contain all the mae value for each epoch, for a certain fold

count = 0

for train_indices, test_indices in k_fold_validator.split(white_features_df):
    
  white_train_features_df = white_features_df.iloc[train_indices, :]
  white_train_features_df_s = pd.DataFrame(scaler.fit_transform(white_train_features_df),
                                             columns=white_train_features_df.columns)

  white_train_targets = white_targets[train_indices]
    
  white_test_features_df = white_features_df.iloc[test_indices, :]
  white_test_features_df_s = pd.DataFrame(scaler.fit_transform(white_test_features_df),
                                             columns=white_test_features_df.columns)

  white_test_targets = white_targets[test_indices]

  model = build_model_wame(white_train_features_df_s, units_number)
  history = fit_model(model, white_train_features_df_s, white_train_targets, white_test_features_df_s, white_test_targets, epochs_number)
  history_dict = history.history
  fold_mae_history = history_dict['val_mae'] # this list contains the value of mae for each epoch, for this fold
  all_folds_mae_history.append(fold_mae_history)
  count += 1
  print('Fold ' + str(count) + ' done.')

In [ ]:
#We plot the results once again

average_mae_history = [np.mean([x[j] for x in all_folds_mae_history]) 
                       for j in range(epochs_number)]

fig, ax = plt.subplots()

ax.plot(range(epochs_number), average_mae_history)
ax.set_xlabel('Epochs')
ax.set_ylabel('Mean absolute error')

In [ ]:
#We now run the sensitivity analysis for the WAMEprop optimiser

learning_rates = [0.0001, 0.0002, 0.0005, 0.001]
units_number = 32
epochs_number = 75 #we will use 75 for the real test
average_mae_history_all_lambdas = []

for learning_rate in learning_rates:

  k_fold_validator = KFold(n_splits=10)
  scaler = StandardScaler()

  all_folds_mae_history = []
  #this list will contain other lists, each of which will contain all the mae value for each epoch, for a certain fold

  count = 0

  for train_indices, test_indices in k_fold_validator.split(white_features_df):
      
    white_train_features_df = white_features_df.iloc[train_indices, :]
    white_train_features_df_s = pd.DataFrame(scaler.fit_transform(white_train_features_df),
                                              columns=white_train_features_df.columns)

    white_train_targets = white_targets[train_indices]
      
    white_test_features_df = white_features_df.iloc[test_indices, :]
    white_test_features_df_s = pd.DataFrame(scaler.fit_transform(white_test_features_df),
                                              columns=white_test_features_df.columns)

    white_test_targets = white_targets[test_indices]

    model = build_model_wame(white_train_features_df_s, units_number, lambda_rate=learning_rate)
    history = fit_model(model, white_train_features_df_s, white_train_targets, white_test_features_df_s, white_test_targets, epochs_number)
    history_dict = history.history
    fold_mae_history = history_dict['val_mae'] # this list contains the value of mae for each epoch, for this fold
    all_folds_mae_history.append(fold_mae_history)
    count += 1
    print('Fold ' + str(count) + ' done.')
  
  average_mae_history = [np.mean([x[j] for x in all_folds_mae_history]) for j in range(epochs_number)]
  average_mae_history_all_lambdas.append(average_mae_history)

  print('Learning rate = ' + str(learning_rate) + ' done.')

In [ ]:
#We now plot the sensitivity analysis results

fig, ax = plt.subplots()

for i in range(len(average_mae_history_all_lambdas)):
  ax.plot(range(epochs_number), average_mae_history_all_lambdas[i])
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Mean absolute error')
ax.legend(['lr=0.0001', 'lr=0.0002', 'lr=0.0005', 'lr=0.001'])